In [1]:
import dagshub
import mlflow
from mlflow.client import MlflowClient
client = MlflowClient(tracking_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow',
                      registry_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow')
import pandas as pd
import numpy as np
import yaml
import os
import json
from pathlib import Path #type: ignore

from src.utils import load_yaml

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import hyperopt
from hyperopt.pyll.base import scope
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')

import mlflow.sklearn
import mlflow.xgboost
import mlflow.pyfunc

In [2]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Raj-Narayanan-B'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8af4cc66be8aec751397fd525e47ae395fa67442'

In [3]:
mlflow.get_tracking_uri()

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [ ]:
client.list_artifacts('c03a395f2448493aa7f2f5c4e066e0d3')

In [ ]:
run_id = '7650b7097cb644d08b8fe61bb3065b15'
filter_string=f"tags.run_type ILIKE 'child'"
runs_df = mlflow.search_runs(experiment_ids=['60'])

In [ ]:
runs_df['artifact_uri'][0]

In [ ]:
best_run_id = mlflow.search_runs(experiment_ids=['60'],
                        filter_string=filter_string,
                        order_by = ['metrics.Cost'])[['run_id','artifact_uri','metrics.Cost']]['run_id'][0]
best_run_id

In [ ]:
mlflow.get_run(f'{best_run_id}')

In [ ]:
artifact_path_name = json.loads(mlflow.get_run(f'{best_run_id}').data.tags['mlflow.log-model.history'])[0]['artifact_path']

In [4]:
train_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_train_data.csv").iloc[:2000,:]
test_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_test_data.csv").iloc[:2000,:]

In [5]:
# x = df.drop(columns='class')
# y = df['class']
# # skf = StratifiedKFold()
# x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state=8)
x_train,y_train = train_df.drop(columns='class'), train_df['class']
x_test,y_test = test_df.drop(columns='class'), test_df['class']

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(2000, 170) (2000, 170) (2000,) (2000,)
class
0    1973
1      27
Name: count, dtype: int64
class
0    1971
1      29
Name: count, dtype: int64


In [ ]:
params = {'hidden_layer_sizes': (500, 300, 200, 150)}

In [ ]:
np.array(params['hidden_layer_sizes'])

In [ ]:
{"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])}

In [ ]:
mlflow.models.infer_signature(x_train, y_test, {"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])})

In [ ]:
params

In [ ]:
mlflow.log_params({"hidden_layer_sizes": [500, 300, 200, 150]})

In [ ]:
mlflow.set_tracking_uri()

In [ ]:
space = {'n_estimators': scope.int(hp.quniform('n_estimators', 100, 150, 1)),
    'criterion': hp.choice('criterion', ['log_loss', 'entropy', 'gini']),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample'])}

mlflow.start_run()

mlflow.set_registry_uri
def objective(space):
    hp_rand_forest = RandomForestClassifier(**space)
    hp_rand_forest.fit(x_train,y_train)
    y_pred = hp_rand_forest.predict(x_test)
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    cost = float((10*fp)+(500*fn))
    return cost

trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 1,
            trials= trials)
best_randf=space_eval(space,best)
best_randf

In [ ]:
import mlflow
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Create base classifiers
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# Log parameters
with mlflow.start_run():
    mlflow.log_params({
        'base_classifiers': base_classifiers,
        'final_estimator': 'DecisionTreeClassifier'
    })

    # Train and log other metrics as needed
    # ...

# Close the MLflow run
mlflow.end_run()


In [ ]:
eval('RandomForestClassifier()')

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

params = {'be1':['rf1','RandomForestClassifier()']}

mlflow.models.infer_signature(x_train, y_test, params)

In [ ]:
with mlflow.start_run(experiment_id = 75):
    mlflow.log_param(key = 'be1',
                     value = {'rf1','RandomForestClassifier()'})

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# # Log parameters
# with mlflow.start_run(experiment_id = 75):
#     mlflow.log_params({
#         'base_classifiers': eval('base_classifiers'),
#         'final_estimator': eval('DecisionTreeClassifier()')
#     })

In [ ]:
params = {
        'base_classifiers': eval('base_classifiers'),
        'final_estimator': eval('DecisionTreeClassifier()')
    }

In [ ]:
for key,value in params.items():
    params[key] = str(value)

params

In [ ]:
mlflow.models.infer_signature(x_train, y_test, new_params)

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]
str(eval('base_classifiers'))

In [ ]:
eval('base_classifiers')

In [ ]:
new_params = client.get_run('ad5d40a9fc33487f8e616dde699ef7ec').data.params

In [ ]:
new_params

In [ ]:
eval("[('KNN_Classifier', KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))]")

In [ ]:
for key,value in new_params.items():
                try:
                    new_params[key] = eval(value)
                except:
                    new_params[key] = value
                    if value == 'nan':
                        new_params[key] = np.nan
new_params                        

In [ ]:
processed_new_params = {key: value for key, value in new_params.items() if value is not None}

In [ ]:
processed_new_params

In [ ]:
stacking_clf_params = stacking_clf.get_params()
for key, value in stacking_clf_params.items():
    stacking_clf_params[key] = str(value)
stacking_clf_params

In [ ]:
exp_id_list = ['84','85','86']
mlflow.search_runs(experiment_ids = exp_id_list,
                            search_all_experiments = True,
                            filter_string = f"tags.run_type ilike 'parent'")

In [ ]:
client.tracking_uri

In [ ]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                  'Stacked_Classifier' : StackingClassifier,
                  'Voting_Classifier' : VotingClassifier
                  }

for key in models.keys():
    try:
        client.delete_registered_model(f"Challenger_{key}")
        client.delete_registered_model(f"Champion {key}")
    except:
        pass

In [ ]:
for i in range(274,278):
    try:
        mlflow.delete_experiment(i)
        print(i)
    except:
        pass

In [ ]:
mlflow.delete_experiment('136389952941582973')

In [ ]:
mlflow.delete_experiment('647179760550514699')

In [ ]:
mlflow.delete_experiment('297515542365458558')

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
for i in range(1,106):
    try:
        os.system(f'mlflow gc --experiment-ids {i}')
        print(i)
    except:
        pass

In [ ]:
[i() for i in models.values()]

In [ ]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  # 'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(),[i() for i in models.values()]))
estimators

In [ ]:
stacking_clf = StackingClassifier(estimators = estimators,
                                  final_estimator = XGBClassifier(),
                                  stack_method = 'auto',
                                  passthrough=True)

In [ ]:
stacking_clf.get_params()

In [ ]:
client.tracking_uri

In [ ]:
client._registry_uri

In [ ]:
stacked_clf_params = stacking_clf.get_params()

In [ ]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params({key: value for key, value in stacked_clf_params.items() if value is not None})

In [ ]:
{key: value for key, value in stacked_clf_params.items() if value is not None}

In [ ]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(), [i() for i in models.values()]))#[value() if key != 'XGB_Classifier' else 'XGBClassifier()' for key,value in models.items()]))
estimators

In [ ]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        xgb_params = {key: value for key, value in estimators[i][1].get_params().items() if value is not None}

xgb_params

In [ ]:
xgb_params

In [ ]:
new_str

In [ ]:
estimators

In [ ]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        estimators[i][1].set_params(**xgb_params)

In [ ]:
str(estimators)

In [ ]:
new_params = {'estimators' : estimators,
              'XGB_Classifier' : 'XGBClassifier()',
              'final_estimator' : 'XGBClassifier()',
              'stack_method' : 'auto',
              'passthrough' : 'True',
              'xgb_params': xgb_params}

In [ ]:
new_params

In [ ]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params(new_params)
    # mlflow.log_param(key = 'XGBClassifier',
    #                  value = XGBClassifier())
    # mlflow.log_param(key = 'final_estimator',
    #                  value = XGBClassifier())

In [ ]:
stacked_run = mlflow.get_run('aaaec579ec5f4a63bad4dd03a8c6f1ae')

In [ ]:
import numpy as np
run_params = {}
for key,value in stacked_run.data.params.items():
    try:
        run_params[key] = eval(value)
    except:
        run_params[key] = value
        if value == 'nan':
                            run_params[key] = np.nan
run_params

In [ ]:
run_params['xgb_params'] = run_params['xgb_params'].replace("nan",'None')
xgb_params_ = eval(run_params['xgb_params'])
xgb_params['missing'] = np.nan
xgb_params

In [ ]:
run_params['estimators'].append(('XGBClassifier',run_params['XGB_Classifier'].set_params(**xgb_params)))

In [ ]:
run_params

In [ ]:
estimators

In [ ]:
s_clf = StackingClassifier(estimators=estimators,
                   final_estimator=XGBClassifier(),
                   cv=5,
                   stack_method = 'auto',
                   passthrough=True)

# estimators,
# final_estimator,
# cv,
# stack_method,
# passthrough
s_clf.get_params()

In [ ]:
s_clf.get_params()['estimators']
s_clf.get_params()['final_estimator']
s_clf.get_params()['cv']
s_clf.get_params()['stack_method']
s_clf.get_params()['passthrough']

In [ ]:
s_clf.get_params()['estimators']

In [ ]:
f"{s_clf.get_params()['final_estimator'].__class__.__name__}_params"

In [ ]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append(s_clf.get_params()['estimators'][i])
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()
        if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
            xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
    xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : 'XGBClassifier()',
                'cv' : 8,#s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}
    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}

    mlflow.log_params(s_clf_params)
    mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))
    mlflow.sklearn.log_model()
    

In [ ]:
s_clf.fit(x_train,y_train)

In [ ]:
s_clf_params_ = s_clf.get_params()
for key, value in s_clf_params_.items():
    s_clf_params_[key] = str(value)
signature = mlflow.models.infer_signature(model_input = x_train,
                                          model_output = s_clf.predict(x_train),
                                          params = s_clf_params_)

In [ ]:
signature

In [ ]:
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
xgb_params

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb_params = {key:value for key,value in xgb.get_params().items() if value is not None}
xgb_params


In [ ]:
s_clf_params

In [ ]:
s_clf_params = {'estimators' : clf_list,
          'final_estimator' : 'XGBClassifier()',
          'cv' : s_clf.get_params()['cv'],
          'stack_method' : s_clf.get_params()['stack_method'],
          'passthrough' : s_clf.get_params()['passthrough'],
          'xgb_params' : xgb_params}

s_clf_params

In [ ]:
trial_params = {'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')), 
                ('Random Forest', RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy', max_features=None, n_estimators=139)), 
                ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier(max_iter=575)), 
                ('Bagging_Classifier', BaggingClassifier(n_estimators=59)), 
                ('Ada_Boost', AdaBoostClassifier(algorithm='SAMME', n_estimators=133)), 
                ('Grad_Boost', GradientBoostingClassifier(criterion='squared_error', loss='exponential', max_features='log2', n_estimators=143))], 
 'final_estimator': 'XGBClassifier()', 
 'cv': 5, 
 'stack_method': 'auto', 
 'passthrough': False, 
 'xgb_params': {'objective': 'binary:logistic', 
                'booster': 'dart', 
                'enable_categorical': False, 
                'learning_rate': 6.5697687477009445, 
                'missing': np.nan, 
                'n_estimators': 164, 
                'tree_method': 'hist'}
}

In [ ]:
trial_params['estimators'].append(('XGB_Classifier',XGBClassifier()))
trial_params

In [ ]:
trial_params['estimators']

In [ ]:
s_clf.get_params()['final_estimator']

In [ ]:
clf_list = []
for i in range(len(trial_params['estimators'])):
    clf_list.append((trial_params['estimators'][i][0],trial_params['estimators'][i][1].__class__()))

estimators_params = {}
for i in range(len(trial_params['estimators'])):
        estimators_params[trial_params['estimators'][i][0]] = trial_params['estimators'][i][1].get_params()
for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
s_clf_params_trial_2 = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
for key,value in estimators_params.items():
        s_clf_params_trial_2[f"{key}_params"] = value
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial_2)

In [ ]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial)

In [ ]:
s_clf = StackingClassifier(estimators = trial_params['estimators'],
                           final_estimator=RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                         max_features=None, n_estimators=139),
                           cv=5,
                           stack_method = 'auto',
                           passthrough=True)

In [ ]:
s_clf

In [ ]:
s_clf.get_params()['estimators'][5][1].__class__()

In [ ]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append((s_clf.get_params()['estimators'][i][0],s_clf.get_params()['estimators'][i][1].__class__()))
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()

    for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        
            s_clf_params = {'estimators' : clf_list,
                    'final_estimator' : 'XGBClassifier()',
                    'cv' : s_clf.get_params()['cv'],
                    'stack_method' : s_clf.get_params()['stack_method'],
                    'passthrough' : s_clf.get_params()['passthrough']}
            for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
        for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params)
#     # mlflow.models.infer_signature(model_input=x_train,
#     #                           model_output = s_clf.predict(x_train))
#     # mlflow.sklearn.log_model()
    

In [ ]:
s_clf_params

In [ ]:
s_clf.get_params()['final_estimator']

In [ ]:
s_clf_params

In [ ]:
s_clf.get_params()['estimators'][0][0]

In [ ]:
estimators_params

In [ ]:
s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier'

In [ ]:
name_of_dictionary = "my_dict"
globals()[name_of_dictionary] = {'hi':1}

In [ ]:
name_of_dictionary

In [ ]:
estimators_params['XGB_Classifier'].items()

In [ ]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(trial_params)

In [ ]:
mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))

In [ ]:
v_clf = VotingClassifier()

In [ ]:
s_clf_run = client.get_run('3f360d91251446b488d0255f44ea1658')

In [ ]:
best_model = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/3f360d91251446b488d0255f44ea1658/artifacts/optuna_Stacked_Classifier')

In [ ]:
best_model.get_params()

In [ ]:
s_clf_run.data.params

In [ ]:
best_model1 = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/5c209d92a1174a5aaefc84258e073304/artifacts/optuna_Stacked_Classifier')
best_model1.get_params()

In [ ]:
import mlflow.xgboost

In [ ]:
xgboost = mlflow.xgboost.load_model('mlflow-artifacts:/1cd799950ea74cae9dbec616b6b65993/acc08000975c49d7a3818110253855f1/artifacts/optuna_XGB_Classifier')

In [ ]:
params = client.get_run('acc08000975c49d7a3818110253855f1').data.params

In [ ]:
params

In [ ]:
with mlflow.start_run(experiment_id='188'):
    mlflow.log_params(params)

In [ ]:
params_ = client.get_run('13ba590fba3f4584ba34cf81006dc31d').data.params

In [ ]:
params_

In [ ]:
rand_f_model = mlflow.sklearn.load_model("mlflow-artifacts:/489d8cb8991a4e58b57565bc00e31b9e/34cc0da17feb4e0ea750551a65809ba8/artifacts/optuna_Random Forest")

In [ ]:
rand_f_model.get_params()

In [ ]:
rand_f_params = client.get_run('34cc0da17feb4e0ea750551a65809ba8').data.params
rand_f_params

In [ ]:
with mlflow.start_run(experiment_id='181'):
    mlflow.log_params(rand_f_params)